In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
from Model import model

In [ ]:
m = Model()
genes_12 = ['CLN3', 'SWI5', 'CLN1', 'CLN2', 'CDH1',  'CDC20', 'CLB5', 'CLB6', 'SIC1', 'CLB1', 'CLB2', 'MCM1']

## Extracting subnetwork based on modularity

In [ ]:
from subnetwork_modularity import grow_subnetwork

subnetwork_nodes, modularity = grow_subnetwork(m.graph, genes_12)
print("number of nodes in subnetwork: ", len(list(subnetwork_nodes)))
print("modularity: ", modularity)

with open('subnetwork_nodes.pkl', 'wb') as f:
    pickle.dump(subnetwork_nodes, f)

## Studying feedback loop

In [ ]:
cycles = []
for gene in (m.graph).predecessors("SWI5"):
    cycles.append(list(nx.all_simple_paths(m.graph, source="SWI5", target=gene, cutoff=5)))

with open('cycles.pkl', 'wb') as f:
    pickle.dump(cycles, f)

In [ ]:
def rank_cycles(loops):
    flat_list = [item for sublist in loops for item in sublist]
    scores = []
    normalization = np.mean(list(nx.get_edge_attributes(m.graph, "alpha").values()))
    for idx, sublist in enumerate(flat_list):
        # score measures the strength of the loop
        score = 1
        previous = sublist[-1]
        for node in sublist:
            score = score * m.graph.get_edge_data(previous, node, default = 0)['alpha'] / normalization
            previous = node
        scores.append([score, idx])
    scores.sort(reverse=True, key= lambda x: abs(x[0]))
    return scores

In [ ]:
rank = rank_cycles(cycles)
flat_list = [item for sublist in cycles for item in sublist]
for i in rank[0:30]:
    print(flat_list[i[1]])

In [ ]:
data = [item[0] for item in rank5]
data_neg = []
data_pos = []
for score in data:
    if score > 0:
        data_pos.append(score)
    if score < 0:
        data_neg.append(abs(score))

plt.figure(figsize=(15,5))
hist, bins = np.histogram(data_pos+data_neg, bins=100)
logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))
plt.hist(data_pos, bins=logbins, alpha=0.8)
plt.hist(data_neg, bins=logbins, alpha=0.6)
plt.legend(["positive feedback", "negative feedback"])
plt.yscale('log')
plt.xscale('log')
plt.xlabel("|gamma| [$minute^{-1}$]")
plt.ylabel("number of occurences")
plt.title("Histogram of the scores of feedback loops affecting node SWI5 \n(with cutoff length of 6)")
plt.savefig("histogram.png", dpi=300)
plt.show()

## Create image of the network

In [ ]:
nodes = ["SWI5", "PRP9", "CTR1", "REE1", "PCL5", "FRE1"]
m = Model(nodes_subgraph=nodes)

m.draw("network")

## Visualize evolution of the system

In [ ]:
m = Model()

lst_index_12_genes = [m.graph.node[element]['index'] for element in genes_12]
lst_index_12_genes

m.cycle(dt= 1, perturbed_gene="SWI5", track = lst_index, num_step = int(1e4))

In [ ]:
m.visualize(legends=genes_12)

In [ ]:
m.visualize_double_log(legends=genes_12)